<a href="https://colab.research.google.com/github/EikEUFS/BuscaOtimizacao/blob/master/otimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-posthocs

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import math
import numpy as np
import matplotlib.pyplot as plt
import statistics as st

from scipy.stats import friedmanchisquare, kruskal, mannwhitneyu, wilcoxon
import scikit_posthocs as sp

In [3]:
'''
n_x      <- size solution       (number)
min_x    <- value min solution  (number)
max_x    <- value max solution  (number)

x        -> solution            (number)
'''
def create_random_vector(n_x, min_x, max_x):
  x = np.random.uniform(min_x, max_x, n_x)
  
  return x

In [4]:
'''
x        <- solution       ([number])
c        <- costs          ([number])
q        <- quantity       (number)

o_x      -> bests solution ([number])
'''
def select_bests_vector(x, c, q):
  o_c = sorted(c)
  o_x = []

  for i in range(q):
      o_x.append(x[c.index(o_c[i])])
      
  return o_x

In [5]:
'''
v        <- solution 1          ([number])
w        <- solution 2          ([number])

n_v      -> new solution 1      ([number])
n_w      -> new solution 2      ([number])
'''
def two_point_crossover(v, w):
  size = min(len(v), len(w))
  c = np.random.randint(0, size)
  d = np.random.randint(0, size - 1)

  aux = np.copy(v)
  n_v = np.copy(v)
  n_w = np.copy(w)  

  if d >= c:
    d += 1
  else:
    c, d = d, c

  n_v[c:d] = n_w[c:d]
  n_w[c:d] = aux[c:d]

  return n_v, n_w

In [6]:
'''
pop_x    <- solutions           ([[number]])
pop_c    <- costs solutions     ([number])
pop_s    <- solutions size      ([number])
to_s     <- tournament size     (number)
f_s      <- function sphere     (function)

b_x      -> best solution      ([number])
b_c      -> best cost solution (number)
'''
def tournament_selection(pop_x, pop_c, pop_s, to_s, f_s):
  b_x = []
  b_c = np.nan
  for i in range(to_s):
      index = np.random.randint(0, pop_s)
      if (np.isnan(b_c) or (pop_c[index] < b_c)):
        b_x = np.copy(pop_x[index])
        b_c = pop_c[index]
        
  return b_x, b_c

In [7]:
'''
x        <- solution            ([number])
n_x      <- size solution       (number)
min_x    <- value min solution  (number)
max_x    <- value max solution  (number)
p        <- probability         (number)
r        <- exploration         (number)

new_x    -> new solution        ([number])
c        -> count               (number)
'''
def tweak_book(x, n_x, min_x, max_x, p, r):
  new_x = np.copy(x)
  c = 0

  for i in range(n_x):
    if (p >= np.random.random_sample()):
      
      while True:
        n = np.random.uniform(r * -1, r)
        if (min_x <=  new_x[i] + n <= max_x):
          new_x[i] = new_x[i] + n
          c += 1
          break

  return new_x, c

In [8]:
'''
x        <- solution            ([number])
n_x      <- size solution       (number)
min_x    <- value min solution  (number)
max_x    <- value max solution  (number)
p        <- probability         (number)
r        <- exploration         (number)

new_x    -> new solution        ([number])
c        -> count               (number)
'''
def tweak_eike(x, n_x, min_x, max_x, p, r):
  new_x = np.copy(x)
  c = 0

  for i in range(n_x):
    if (p > np.random.random_sample()):
      new_x[i] = np.random.uniform(min_x, max_x)
      c += 1
      
  return new_x, c

In [9]:
'''
x        <- solution  ([number])
o        <- optimizer ([number])
b        <- bias      (number)

c        -> cost      (number)
'''
def shifted_sphere(x, o, b):
  z = x - o
  c = np.sum(z**2) + b
  
  return c

In [10]:
'''
x        <- solution  ([number])
o        <- optimizer ([number])
b        <- bias      (number)

c        -> cost      (number)
'''
def shifted_rosenbrock(x, o, b):
  z = x - o + 1
  c = np.sum(100.0*(z[1:]-z[:-1]**2.0)**2.0 + (1-z[:-1])**2.0) + b
  
  return c

In [11]:
'''
x        <- solution  ([number])
o        <- optimizer ([number])
b        <- bias      (number)

c        -> cost      (number)
'''
def shifted_rastring(x, o, b):
  z = x - o
  c = np.sum(z**2.0 - 10 * np.cos(2 * np.pi * z) + 10) + b

  return c

In [12]:
'''
n_x      <- size solution       (number)
min_x    <- value min solution  (number)
max_x    <- value max solution  (number)
p        <- probability         (number)
r        <- exploration         (number)
a        <- avaliation          (number)
o        <- optimizer           ([number])
b        <- bias                (number)
f_t      <- function tweak      (function)
f_s      <- function sphere     (function)

cs       -> costs               ([number])
'''
def hill_climbing(n_x, min_x, max_x, p, r, a, o, b, f_t, f_s):
  x = create_random_vector(n_x, min_x, max_x)   # solution ([number])
  cs = []                                       # costs ([number])

  b_x = np.copy(x)                              # best_solution ([number])
  b_c = f_s(b_x, o, b)                          # best_cost (number)

  for _ in range(a):
    c_x, _ = f_t(b_x, n_x, min_x, max_x, p, r)  # candidate_solution ([number])
    c_c = f_s(c_x, o, b)                        # candidate_cost (number)

    if (c_c <= b_c):
      b_x = np.copy(c_x)
      b_c = c_c    

    cs.append(b_c)
    
  return cs

In [13]:
'''
n_x      <- size solution       (number)
min_x    <- value min solution  (number)
max_x    <- value max solution  (number)
p        <- probability         (number)
r        <- exploration         (number)
a        <- avaliation          (number)
o        <- optimizer           ([number])
b        <- bias                (number)
f_t      <- function tweak      (function)
f_s      <- function sphere     (function)

cs       -> costs               ([number])
'''
def simulated_annealing(n_x, min_x, max_x, p, r, a, o, b, f_t, f_s):
  x = create_random_vector(n_x, min_x, max_x)   # solution ([number])
  cs = []                                       # costs ([number])

  i_x = np.copy(x)                              # initial_solution ([number])
  i_c = f_s(i_x, o, b)                          # initial_cost (number)

  b_x = np.copy(i_x)                            # best_solution ([number])
  b_c = i_c                                     # best_cost (number)

  for i in range(a):
    f = i / float(a)                            # fraction (number)
    t = max(0.01, min(1, 1 - f))                # temperature (number)

    c_x, _  = f_t(i_x, n_x, min_x, max_x, p, r) # candidate_solution ([number])
    c_c     = f_s(c_x, o, b)                    # candidate_cost (number)

    if ((c_c <= i_c) or (np.random.random_sample() < np.exp(- (c_c - i_c) / t))):
      i_x = np.copy(c_x)
      i_c = c_c

    if (i_c <= b_c):
      b_x = np.copy(i_x)
      b_c = i_c

    cs.append(b_c)

  return cs

In [14]:
'''
n_x      <- size solution       (number)
min_x    <- value min solution  (number)
max_x    <- value max solution  (number)
p        <- probability         (number)
r        <- exploration         (number)
a        <- avaliation          (number)
o        <- optimizer           ([number])
b        <- bias                (number)
t_s      <- tabu size           (number)
f_t      <- function tweak      (function)
f_s      <- function sphere     (function)

cs       -> costs               ([number])
'''
def search_tabu(n_x, min_x, max_x, p, r, a, o, b, t_s, f_t, f_s):
  x = create_random_vector(n_x, min_x, max_x)   # solution ([number])
  cs = []                                       # costs ([number])
  t = []                                        # tabu ([number])

  i_x = np.copy(x)                              # initial_solution ([number])
  i_c = f_s(i_x, o, b)                          # initial_cost (number)

  b_x = np.copy(i_x)                            # best_solution ([number])
  b_c = i_c                                     # best_cost (number)

  for i in range(a):
    if (len(t) > t_s):
      t = np.copy(t[1:])

    c_x, c = f_t(i_x, n_x, min_x, max_x, p, r)  # candidate_solution, value tabu ([number], number)

    while (c in t):
      c_x, c = f_t(i_x, n_x, min_x, max_x, p, r)
      
    c_c = f_s(c_x, o, b)                        # candidate_cost (number)

    t = np.append(t, c)

    if (c_c <= i_c):
      i_x = np.copy(c_x)
      i_c = c_c

    if (i_c <= b_c):
      b_x = np.copy(i_x)
      b_c = i_c

    cs.append(b_c)
  
  return cs

In [15]:
'''
n_x      <- size solution       (number)
min_x    <- value min solution  (number)
max_x    <- value max solution  (number)
p        <- probability         (number)
r        <- exploration         (number)
a        <- avaliation          (number)
o        <- optimizer           ([number])
b        <- bias                (number)
pa_s     <- parents size        (number)
ch_s     <- childrens size      (number)
f_t      <- function tweak      (function)
f_s      <- function sphere     (function)

cs       -> costs               ([number])
'''
def evolution_strategy(n_x, min_x, max_x, p, r, a, o, b, pa_s, ch_s, f_t, f_s):
  cs = []                                         # costs ([number])

  pop_x = []                                      # solution       ([[number]])
  pop_c = []                                      # costs solution ([number])

  for _ in range(ch_s):
    pop_x.append(np.copy(create_random_vector(n_x, min_x, max_x)))

  b_x = []                                        # best_solution ([number])
  b_c = np.nan                                    # best_cost (number)

  for _ in range(a):
    for i in range(ch_s):
      pop_c.append(f_s(pop_x[i], o, b))

      if (np.isnan(b_c) or (pop_c[i] < b_c)):
        b_x = np.copy(pop_x[i])
        b_c = pop_c[i]
  
    q_x = np.copy(select_bests_vector(pop_x, pop_c, pa))   # selected bests solutions ([[number]])

    pop_x = np.copy(q_x)
    pop_x = pop_x.tolist()
    pop_c = []

    q_ch = int(ch_s/pa_s)                                  # quantity new childrens (number)

    for i in range(pa_s):
      for _ in range(q_ch):
        c_x, _ = f_t(q_x[i], n_x, min_x, max_x, p, r)
        pop_x.append(c_x)

    cs.append(b_c)
  
  return cs

In [16]:
'''
n_x      <- size solution       (number)
min_x    <- value min solution  (number)
max_x    <- value max solution  (number)
p        <- probability         (number)
r        <- exploration         (number)
a        <- avaliation          (number)
o        <- optimizer           ([number])
b        <- bias                (number)
e_s      <- elite size          (number)
pop_s    <- solutions size      (number)
to_s     <- tournament size     (number)
f_t      <- function tweak      (function)
f_s      <- function sphere     (function)

cs       -> costs               ([number])
'''
def genetic_algorithm(n_x, min_x, max_x, p, r, a, o, b, e_s, pop_s, to_s, f_t, f_s):
  cs = []                                         # costs ([number])

  pop_x = []                                      # solution       ([[number]])
  pop_c = []                                      # costs solution ([number])

  for _ in range(pop_s):
    pop_x.append(np.copy(create_random_vector(n_x, min_x, max_x)))  

  b_x = []                                        # best_solution ([number])
  b_c = np.nan                                    # best_cost (number)

  for _ in range(a):
    for i in range(pop_s):  
      pop_c.append(f_s(pop_x[i], o, b))

      if (np.isnan(b_c) or (pop_c[i] < b_c)):
        b_x = np.copy(pop_x[i])
        b_c = pop_c[i]
  
    q_x = np.copy(select_bests_vector(pop_x, pop_c, e_s))   # selected bests solutions ([[number]])
    q_x = q_x.tolist()    

    q_pop = int((pop_s - e_s)/2)

    for i in range(q_pop):
      pa_x, _ = np.copy(tournament_selection(pop_x, pop_c, pop_s, to_s, f_s))
      pb_x, _ = np.copy(tournament_selection(pop_x, pop_c, pop_s, to_s, f_s))
      ca_x, cb_x = two_point_crossover(pa_x, pb_x)
      ma_x, _ = np.copy(f_t(ca_x, n_x, min_x, max_x, p, r))
      mb_x, _ = np.copy(f_t(cb_x, n_x, min_x, max_x, p, r))
      q_x.append(np.copy(ma_x))
      q_x.append(np.copy(mb_x))

    pop_x = np.copy(q_x)
    pop_c = []

    cs.append(b_c)
  
  return cs

In [17]:
def test_optimization():
  n_x = 100       # Size Array
  min_x = -100    # Min X
  max_x = 100     # Max X
  a = 50000       # Avaliation
  s = 10          # Repeat

  # BIAS e Otimo do Sphere
  o_sp = np.array([
                  -65,-2,-21,86,65,-27,-91,7,86,49,-58,96,-77,10,86,-78,48,13,-28,63,95,-58,-96,7,-21,
                  -100,42,8,-67,56,-99,18,34,-67,-46,-14,78,-98,34,29,-47,16,62,26,-20,-94,-7,87,-32,-7,
                  43,-36,-90,95,84,60,98,0,82,-21,9,-84,57,-61,50,84,64,-82,-24,-85,-80,82,-63,-94,19,-43,
                  -100,-62,3,-34,-83,12,-68,-43,95,-68,-8,69,89,-37,64,-35,-59,-83,-67,-93,28,-76,-26,-65
                  ])
  b_sp = -450
    
  # BIAS e Otimo do Rosenbrock
  o_ro = np.array([
                  -35.91,-69.14,4.93,4.43,42.76,0.85,-15.64,68.11,6.99,9.92,5.88,-0.18,18.55,-0.87,5.79,
                  -5.18,4.78, 35.42,-0.56,-41.91,5.23,58.15,69.25,-54.05,30.73,15.73,-40.22,-8.71,48.12,
                  43.39,23.15,-8.33,-1.09,-46.20,-32.94,-37.73,-1.11,-0.49,-2.40,-64.44,-4.86,-14.40,
                  -4.35,-1.85,-23.65,4.46,-7.11,11.29,0.88,-98.82,79.81,-49.91,26.41,3.51,33.25,3.48,
                  -7.29,0.00,16.88, 6.11,-29.38,-13.48,43.39,-46.88,39.00,0.47,-58.69,-12.16,-3.95,35.87,
                  -6.73,20.23,-13.22,-13.48,66.63,27.15,-36.15,46.46,13.23,-61.69,-36.08,-19.79,-1.00,
                  16.72,-16.19,-38.64,88.76,45.85,-33.61,11.32,-46.06,-31.68,20.94,29.43,32.87,-17.49,
                  -20.12,-13.16,-0.86, 20.54
                  ])
  b_ro = 390

  # BIAS e Otimo do Rastring
  o_ra = np.array([
                  -0.37,-0.79,1.60,0.07,4.77,-1.58,0.16,-2.98,-0.26,3.98,0.93,1.93,-0.72,0.76,0.26,3.23,
                  -1.88,-0.24,0.00,0.36,0.00,2.00,-1.29,2.58,0.51,0.75,-0.62,0.00,0.40,0.01,0.81,-2.36,
                  -0.86,-1.27,0.66,-3.90,-0.96,1.66,-1.49,-1.47,-1.48,1.68,1.46,-0.34,-0.90,1.56,0.00,
                  -1.69,0.30,1.42,-2.59,-0.34,0.77,0.83,-0.19,1.95,-2.07,0.89,-0.47,0.75,0.31,-0.90,0.00,
                  -2.27,-1.37,-2.02,-1.13,3.27,1.27,3.07,-4.30,-2.94,-2.45,0.80,-0.11,0.00,0.00,-1.25,
                  -0.38,2.46,-0.34,0.00,-0.35,1.97,-1.90,2.61,0.00,0.00,0.00,-1.08,-1.67,1.85,-0.94,2.51,
                  2.63,0.84,1.66,2.30,-0.03,-1.06
                  ])
  b_ra = -330

  c_m_cs_h_c_te_sp = []
  c_m_cs_h_c_tb_sp = []
  c_m_cs_h_c_tb_ro = []
  c_m_cs_h_c_tb_ra = []
  c_m_cs_s_a_tb_sp = []
  c_m_cs_s_a_tb_ro = []
  c_m_cs_s_a_tb_ra = []
  c_m_cs_s_t_tb_sp = []
  c_m_cs_s_t_tb_ro = []
  c_m_cs_s_t_tb_ra = []
  c_m_cs_e_s_tb_sp = []
  c_m_cs_e_s_tb_ro = []
  c_m_cs_e_s_tb_ra = []
  c_m_cs_g_a_tb_sp = []
  c_m_cs_g_a_tb_ro = []
  c_m_cs_g_a_tb_ra = []

  for i in range(s):

    cs_h_c_te_sp = hill_climbing(n_x, min_x, max_x, 0.02, 10, a, o_sp, b_sp, tweak_eike, shifted_sphere)
    cs_h_c_tb_sp = hill_climbing(n_x, min_x, max_x, 0.02, 10, a, o_sp, b_sp, tweak_book, shifted_sphere)
    cs_h_c_tb_ro = hill_climbing(n_x, min_x, max_x, 0.02, 10, a, o_ro, b_ro, tweak_book, shifted_rosenbrock)
    cs_h_c_tb_ra = hill_climbing(n_x, min_x, max_x, 0.02, 10, a, o_ra, b_ra, tweak_book, shifted_rastring)
    cs_s_a_tb_sp = simulated_annealing(n_x, min_x, max_x, 0.02, 10, a, o_sp, b_sp, tweak_book, shifted_sphere)
    cs_s_a_tb_ro = simulated_annealing(n_x, min_x, max_x, 0.02, 10, a, o_ro, b_ro, tweak_book, shifted_rosenbrock)
    cs_s_a_tb_ra = simulated_annealing(n_x, min_x, max_x, 0.02, 10, a, o_ra, b_ra, tweak_book, shifted_rastring)
    cs_s_t_tb_sp = search_tabu(n_x, min_x, max_x, 0.02, 10, a, o_sp, b_sp, 3, tweak_book, shifted_sphere)
    cs_s_t_tb_ro = search_tabu(n_x, min_x, max_x, 0.02, 20, a, o_ro, b_ro, 3, tweak_book, shifted_rosenbrock)
    cs_s_t_tb_ra = search_tabu(n_x, min_x, max_x, 0.02, 10, a, o_ra, b_ra, 3, tweak_book, shifted_rastring)
    cs_e_s_tb_sp = evolution_strategy(n_x, min_x, max_x, 0.08, 10, a, o_sp, b_sp, 4, 8, tweak_book, shifted_sphere)
    cs_e_s_tb_ro = evolution_strategy(n_x, min_x, max_x, 0.02, 10, a, o_ro, b_ro, 6, 12, tweak_book, shifted_rosenbrock)
    cs_e_s_tb_ra = evolution_strategy(n_x, min_x, max_x, 0.02, 10, a, o_ra, b_ra, 6, 12, tweak_book, shifted_rastring) 
    cs_g_a_tb_sp = genetic_algorithm(100, -5, 5, 0.02, 10, a, o_sp, b_sp, 2, 8, 2, tweak_book, shifted_sphere)
    cs_g_a_tb_ro = genetic_algorithm(100, -5, 5, 0.02, 10, a, o_ro, b_ro, 2, 4, 2, tweak_book, shifted_rosenbrock)
    cs_g_a_tb_ra = genetic_algorithm(100, -5, 5, 0.02, 10, a, o_ra, b_ra, 6, 12, 2, tweak_book, shifted_rastring)

    c_m_cs_h_c_te_sp.append(cs_h_c_te_sp[-1])
    c_m_cs_h_c_tb_sp.append(cs_h_c_tb_sp[-1])
    c_m_cs_h_c_tb_ro.append(cs_h_c_tb_ro[-1])
    c_m_cs_h_c_tb_ra.append(cs_h_c_tb_ra[-1])
    c_m_cs_s_a_tb_sp.append(cs_s_a_tb_sp[-1])
    c_m_cs_s_a_tb_ro.append(cs_s_a_tb_ro[-1])
    c_m_cs_s_a_tb_ra.append(cs_s_a_tb_ra[-1])
    c_m_cs_s_t_tb_sp.append(cs_s_t_tb_sp[-1])
    c_m_cs_s_t_tb_ro.append(cs_s_t_tb_ro[-1])
    c_m_cs_s_t_tb_ra.append(cs_s_t_tb_ra[-1])
    c_m_cs_e_s_tb_sp.append(cs_e_s_tb_sp[-1])
    c_m_cs_e_s_tb_ro.append(cs_e_s_tb_ro[-1])
    c_m_cs_e_s_tb_ra.append(cs_e_s_tb_ra[-1])
    c_m_cs_g_a_tb_sp.append(cs_g_a_tb_sp[-1])
    c_m_cs_g_a_tb_ro.append(cs_g_a_tb_ro[-1])
    c_m_cs_g_a_tb_ra.append(cs_g_a_tb_ra[-1])

  np_c_m_cs_h_c_te_sp = np.array(c_m_cs_h_c_te_sp)
  np_c_m_cs_h_c_tb_sp = np.array(c_m_cs_h_c_tb_sp)
  np_c_m_cs_h_c_tb_ro = np.array(c_m_cs_h_c_tb_ro)
  np_c_m_cs_h_c_tb_ra = np.array(c_m_cs_h_c_tb_ra)
  np_c_m_cs_s_a_tb_sp = np.array(c_m_cs_s_a_tb_sp)
  np_c_m_cs_s_a_tb_ro = np.array(c_m_cs_s_a_tb_ro)
  np_c_m_cs_s_a_tb_ra = np.array(c_m_cs_s_a_tb_ra)
  np_c_m_cs_s_t_tb_sp = np.array(c_m_cs_s_t_tb_sp)
  np_c_m_cs_s_t_tb_ro = np.array(c_m_cs_s_t_tb_ro)
  np_c_m_cs_s_t_tb_ra = np.array(c_m_cs_s_t_tb_ra)
  np_c_m_cs_e_s_tb_sp = np.array(c_m_cs_e_s_tb_sp)
  np_c_m_cs_e_s_tb_ro = np.array(c_m_cs_e_s_tb_ro)
  np_c_m_cs_e_s_tb_ra = np.array(c_m_cs_e_s_tb_ra)
  np_c_m_cs_g_a_tb_sp = np.array(c_m_cs_g_a_tb_sp)
  np_c_m_cs_g_a_tb_ro = np.array(c_m_cs_g_a_tb_ro)
  np_c_m_cs_g_a_tb_ra = np.array(c_m_cs_g_a_tb_ra)

  
  print('Hill Climbing - Tweak EikE - Shifted Sphere')
  print('p=0.02 | r=10')
  print('Values = ', end="")
  print(np_c_m_cs_h_c_te_sp)
  print('Media =', np_c_m_cs_h_c_te_sp.mean())  
  print('Standard Seviation =', np_c_m_cs_h_c_te_sp.std())
  print()
  print('Hill Climbing - Tweak Book - Shifted Sphere')
  print('p=0.02 | r=10')
  print('Values = ', end="")
  print(np_c_m_cs_h_c_tb_sp)
  print('Media =', np_c_m_cs_h_c_tb_sp.mean())  
  print('Standard Seviation =', np_c_m_cs_h_c_tb_sp.std())
  print()
  print('Hill Climbing - Tweak Book - Shifted Rosenbrock')
  print('p=0.02 | r=10')
  print('Values = ', end="")
  print(np_c_m_cs_h_c_tb_ro)
  print('Media =', np_c_m_cs_h_c_tb_ro.mean())  
  print('Standard Seviation =', np_c_m_cs_h_c_tb_ro.std())
  print()
  print('Hill Climbing - Tweak Book - Shifted Rastring')
  print('p=0.02 | r=10')
  print('Values = ', end="")
  print(np_c_m_cs_h_c_tb_ra)
  print('Media =', np_c_m_cs_h_c_tb_ra.mean())  
  print('Standard Seviation =', np_c_m_cs_h_c_tb_ra.std())
  print()

  print('Simulated Annealing - Tweak Book - Shifted Sphere')
  print('p=0.02 | r=10')
  print('Values = ', end="")
  print(np_c_m_cs_s_a_tb_sp)
  print('Media =', np_c_m_cs_s_a_tb_sp.mean())  
  print('Standard Seviation =', np_c_m_cs_s_a_tb_sp.std())
  print()
  print('Simulated Annealing - Tweak Book - Shifted Rosenbrock')
  print('p=0.02 | r=10')
  print('Values = ', end="")
  print(np_c_m_cs_s_a_tb_ro)
  print('Media =', np_c_m_cs_s_a_tb_ro.mean())  
  print('Standard Seviation =', np_c_m_cs_s_a_tb_ro.std())
  print()
  print('Simulated Annealing - Tweak Book - Shifted Rastring')
  print('p=0.02 | r=10')
  print('Values = ', end="")
  print(np_c_m_cs_s_a_tb_ra)
  print('Media =', np_c_m_cs_s_a_tb_ra.mean())  
  print('Standard Seviation =', np_c_m_cs_s_a_tb_ra.std())
  print()

  print('Search Tabu - Tweak Book - Shifted Sphere')
  print('p=0.02 | r=10 | s_t=3')
  print('Values = ', end="")
  print(np_c_m_cs_s_t_tb_sp)
  print('Media =', np_c_m_cs_s_t_tb_sp.mean())  
  print('Standard Seviation =', np_c_m_cs_s_t_tb_sp.std())
  print()
  print('Search Tabu - Tweak Book - Shifted Rosenbrock')
  print('p=0.02 | r=20 | s_t=3')
  print('Values = ', end="")
  print(np_c_m_cs_s_t_tb_ro)
  print('Media =', np_c_m_cs_s_t_tb_ro.mean())  
  print('Standard Seviation =', np_c_m_cs_s_t_tb_ro.std())
  print()
  print('Search Tabu - Tweak Book - Shifted Rastring')
  print('p=0.02 | r=10 | s_t=3')
  print('Values = ', end="")
  print(np_c_m_cs_s_t_tb_ra)
  print('Media =', np_c_m_cs_s_t_tb_ra.mean())  
  print('Standard Seviation =', np_c_m_cs_s_t_tb_ra.std())
  print()

  print('Evolution Strategy - Tweak Book - Shifted Sphere')
  print('p=0.08 | r=10 | pa=4 | ch=8')
  print('Values = ', end="")
  print(np_c_m_cs_e_s_tb_sp)
  print('Media =', np_c_m_cs_e_s_tb_sp.mean())  
  print('Standard Seviation =', np_c_m_cs_e_s_tb_sp.std())
  print()
  print('Evolution Strategy - Tweak Book - Shifted Rosenbrock')
  print('p=0.02 | r=10 | pa=6 | ch=12')
  print('Values =', end="")
  print(np_c_m_cs_e_s_tb_ro)
  print('Media = ', np_c_m_cs_e_s_tb_ro.mean())  
  print('Standard Seviation =', np_c_m_cs_e_s_tb_ro.std())
  print()
  print('Evolution Strategy - Tweak Book - Shifted Rastring')
  print('p=0.02 | r=10 | pa=6 | ch=12')
  print('Values = ', end="")
  print(np_c_m_cs_e_s_tb_ra)
  print('Media =', np_c_m_cs_e_s_tb_ra.mean())  
  print('Standard Seviation =', np_c_m_cs_e_s_tb_ra.std())
  print()
  
  print('Genetic Algorithm - Tweak Book - Shifted Sphere')
  print('p=0.02 | r=10 | e_s=2 | pop_s=8')
  print('Values = ', end="")
  print(np_c_m_cs_g_a_tb_sp)
  print('Media =', np_c_m_cs_g_a_tb_sp.mean())  
  print('Standard Seviation =', np_c_m_cs_g_a_tb_sp.std())
  print()
  print('Genetic Algorithm - Tweak Book - Shifted Rosenbrock')
  print('p=0.02 | r=10 | e_s=2 | pop_s=4')
  print('Values =', end="")
  print(np_c_m_cs_g_a_tb_ro)
  print('Media = ', np_c_m_cs_g_a_tb_ro.mean())  
  print('Standard Seviation =', np_c_m_cs_g_a_tb_ro.std())
  print()
  print('Genetic Algorithm - Tweak Book - Shifted Rastring')
  print('p=0.02 | r=10 | e_s=6 | pop_s=12')
  print('Values = ', end="")
  print(np_c_m_cs_g_a_tb_ra)
  print('Media =', np_c_m_cs_g_a_tb_ra.mean())  
  print('Standard Seviation =', np_c_m_cs_g_a_tb_ra.std())
  print()

In [18]:
def test_parameters():
  # BIAS e Otimo do Sphere
  o_sp = np.array([
                  -65,-2,-21,86,65,-27,-91,7,86,49,-58,96,-77,10,86,-78,48,13,-28,63,95,-58,-96,7,-21,
                  -100,42,8,-67,56,-99,18,34,-67,-46,-14,78,-98,34,29,-47,16,62,26,-20,-94,-7,87,-32,-7,
                  43,-36,-90,95,84,60,98,0,82,-21,9,-84,57,-61,50,84,64,-82,-24,-85,-80,82,-63,-94,19,-43,
                  -100,-62,3,-34,-83,12,-68,-43,95,-68,-8,69,89,-37,64,-35,-59,-83,-67,-93,28,-76,-26,-65
                  ])
  b_sp = -450
    
  # BIAS e Otimo do Rosenbrock
  o_ro = np.array([
                  -35.91,-69.14,4.93,4.43,42.76,0.85,-15.64,68.11,6.99,9.92,5.88,-0.18,18.55,-0.87,5.79,
                  -5.18,4.78, 35.42,-0.56,-41.91,5.23,58.15,69.25,-54.05,30.73,15.73,-40.22,-8.71,48.12,
                  43.39,23.15,-8.33,-1.09,-46.20,-32.94,-37.73,-1.11,-0.49,-2.40,-64.44,-4.86,-14.40,
                  -4.35,-1.85,-23.65,4.46,-7.11,11.29,0.88,-98.82,79.81,-49.91,26.41,3.51,33.25,3.48,
                  -7.29,0.00,16.88, 6.11,-29.38,-13.48,43.39,-46.88,39.00,0.47,-58.69,-12.16,-3.95,35.87,
                  -6.73,20.23,-13.22,-13.48,66.63,27.15,-36.15,46.46,13.23,-61.69,-36.08,-19.79,-1.00,
                  16.72,-16.19,-38.64,88.76,45.85,-33.61,11.32,-46.06,-31.68,20.94,29.43,32.87,-17.49,
                  -20.12,-13.16,-0.86, 20.54
                  ])
  b_ro = 390

  # BIAS e Otimo do Rastring
  o_ra = np.array([
                  -0.37,-0.79,1.60,0.07,4.77,-1.58,0.16,-2.98,-0.26,3.98,0.93,1.93,-0.72,0.76,0.26,3.23,
                  -1.88,-0.24,0.00,0.36,0.00,2.00,-1.29,2.58,0.51,0.75,-0.62,0.00,0.40,0.01,0.81,-2.36,
                  -0.86,-1.27,0.66,-3.90,-0.96,1.66,-1.49,-1.47,-1.48,1.68,1.46,-0.34,-0.90,1.56,0.00,
                  -1.69,0.30,1.42,-2.59,-0.34,0.77,0.83,-0.19,1.95,-2.07,0.89,-0.47,0.75,0.31,-0.90,0.00,
                  -2.27,-1.37,-2.02,-1.13,3.27,1.27,3.07,-4.30,-2.94,-2.45,0.80,-0.11,0.00,0.00,-1.25,
                  -0.38,2.46,-0.34,0.00,-0.35,1.97,-1.90,2.61,0.00,0.00,0.00,-1.08,-1.67,1.85,-0.94,2.51,
                  2.63,0.84,1.66,2.30,-0.03,-1.06
                  ])
  b_ra = -330

  n_x = 100       # Size Array
  min_x = -5    # Min X
  max_x = 5     # Max X
  a = 50000       # Avaliation
  s = 5           # Repeat

  p = [0.02, 0.08, 0.20]
  r = [10, 20, 50]
  ta_s = [2, 3, 5]
  pa_s = [2, 4, 6, 20]
  ch_s = [4, 8, 12, 40]
  e_s = [2, 4, 6, 20]
  pop_s = [4, 8, 12, 40]
  to_s = 2    

  f_s = [shifted_rastring]
  o = [o_ra]
  b = [b_ra]

  # Hill Climbing
  for l in range(len(f_s)):
    for i in range(len(p)):
      for j in range(len(r)):      
        c_m_cs = []
        for m in range(s):
          cs = hill_climbing(n_x, min_x, max_x, p[i], r[j], a, o[l], b[l], tweak_book, f_s[l])
          c_m_cs.append(cs[-1])

        np_c_m_cs = np.array(c_m_cs)

        print('Simulated Annealing - ', end="")
        if (l == 0):
          print('Shifted Sphere')
        if (l == 1):
          print('Shifted Rosenbrock')
        if (l == 2):
          print('Shifted Rastring')
        print('p = ', p[i],' | r = ', r[j])
        print('Values = ', end="")
        print(np_c_m_cs)
        print('Media =', np_c_m_cs.mean())  
        print('Standard Seviation =', np_c_m_cs.std())
        print()
        print('------------------------------------------------------------------')
        print()

  # Simulated Annealing
  for l in range(len(f_s)):
    for i in range(len(p)):
      for j in range(len(r)):      
        c_m_cs = []
        for m in range(s):
          cs = simulated_annealing(n_x, min_x, max_x, p[i], r[j], a, o[l], b[l], tweak_book, f_s[l])
          c_m_cs.append(cs[-1])

        np_c_m_cs = np.array(c_m_cs)

        print('Simulated Annealing - ', end="")
        if (l == 0):
          print('Shifted Sphere')
        if (l == 1):
          print('Shifted Rosenbrock')
        if (l == 2):
          print('Shifted Rastring')
        print('p = ', p[i],' | r = ', r[j])
        print('Values = ', end="")
        print(np_c_m_cs)
        print('Media =', np_c_m_cs.mean())  
        print('Standard Seviation =', np_c_m_cs.std())
        print()
        print('------------------------------------------------------------------')
        print()

  # Search Tabu
  for l in range(len(f_s)):
    for i in range(len(p)):
      for j in range(len(r)):
        for n in range(len(ta_s)):  
          c_m_cs = []
          for m in range(s):
            cs = search_tabu(n_x, min_x, max_x, p[i], r[j], a, o[l], b[l], ta_s[n], tweak_book, f_s[l])
            c_m_cs.append(cs[-1])

          np_c_m_cs = np.array(c_m_cs)

          print('Tabu Search - ', end="")
          if (l == 0):
            print('Shifted Sphere')
          if (l == 1):
            print('Shifted Rosenbrock')
          if (l == 2):
            print('Shifted Rastring')
          print('p = ', p[i],' | r = ', r[j] , ' | ta_s = ', ta_s[n])
          print('Values = ', end="")
          print(np_c_m_cs)
          print('Media =', np_c_m_cs.mean())  
          print('Standard Seviation =', np_c_m_cs.std())
          print()
          print('------------------------------------------------------------------')
          print()
  
  # Evolution Strategy
  for l in range(len(f_s)):
    for i in range(len(p)):
      for j in range(len(r)):
        for n in range(len(pa_s)): 
          for o in range(len(ch_s)):
            if (pa_s[n] < ch_s[o]):
              c_m_cs = []
              for m in range(s):
                cs = evolution_strategy(n_x, min_x, max_x, p[i], r[j], a, o[l], b[l], pa_s[n], ch_s[o], tweak_book, f_s[l])
                c_m_cs.append(cs[-1])

              np_c_m_cs = np.array(c_m_cs)

              print('Evolution Strategy - ', end="")
              if (l == 0):
                print('Shifted Sphere')
              if (l == 1):
                print('Shifted Rosenbrock')
              if (l == 2):
                print('Shifted Rastring')
              print('p = ', p[i],' | r = ', r[j] , ' | pa_s = ', pa_s[n], ' | ch_s = ', ch_s[n])
              print('Values = ', end="")
              print(np_c_m_cs)
              print('Media =', np_c_m_cs.mean())  
              print('Standard Seviation =', np_c_m_cs.std())
              print()
              print('------------------------------------------------------------------')
              print()

  # Genetic Algorithm
  for count_fs in range(len(f_s)):
    for count_p in range(len(p)):
      for count_r in range(len(r)):
        for count_es in range(len(e_s)): 
          for count_pop_s in range(len(pop_s)):
            if (e_s[count_es] < pop_s[count_pop_s]):
              c_m_cs = []
              for m in range(s):
                cs = genetic_algorithm(n_x, min_x, max_x, p[count_p], r[count_r], a, o[count_fs], 
                                       b[count_fs], e_s[count_es], pop_s[count_pop_s], to_s, 
                                       tweak_book, f_s[count_fs])
                c_m_cs.append(cs[-1])

              np_c_m_cs = np.array(c_m_cs)

              print('Genetic Algorithm - ', end="")
              if (count_fs == 0):
                print('Shifted Sphere')
              if (count_fs == 1):
                print('Shifted Rosenbrock')
              if (count_fs == 2):
                print('Shifted Rastring')
              print('p = ', p[count_p],' | r = ', r[count_r] , ' | e_s = ', e_s[count_es], 
                    ' | pop_s = ', pop_s[count_pop_s], ' | to_s = ', to_s)
              print('Values = ', end="")
              print(np_c_m_cs)
              print('Media =', np_c_m_cs.mean())  
              print('Standard Seviation =', np_c_m_cs.std())
              print()
              print('------------------------------------------------------------------')
              print()

In [19]:
'''
data      <- values best costs       ([number)
'''
def test_statistical(data):
  print_friedman = friedmanchisquare(*(data[i, :] for i in range(data.shape[0])))
  print_kruskal = kruskal(*(data[i, :] for i in range(data.shape[0])))
  print_posthoc = sp.posthoc_conover(data)

  print_posthoc.columns = ['HC', 'SA', 'TABU', 'ES', 'GA']
  print_posthoc.index = ['HC', 'SA', 'TABU', 'ES', 'GA']

  print('FRIEDMAN:')
  print(print_friedman)
  print()
  print('print_kruskal:')
  print(print_kruskal)
  print()
  print('POSTHOC:')
  print(print_posthoc)
  print()
  print()
  print(print_posthoc.to_latex(decimal=",", float_format="%.2f"))

# Resultados








## Hill Climbing

### Tweak EikE - Shifted Sphere
```
p = 0.02
r = 10
v_h_c_t_e_s_s = [ -361.89711511, -333.66210101, -335.64896223, -331.70957379, -360.16050644,
                  -330.50664702, -331.2861224,  -354.40371762, -331.17329636, -334.54814754]
m_h_c_t_e_s_s = -340.49961895211624
d_h_c_t_e_s_s = 12.216468500815258
```

### Tweak Book - Shifted Sphere
```
p = 0.02
r = 10
v_h_c_t_b_s_s = [ -448.99958059, -449.08588058, -448.67478445, -448.8023115,  -448.88409216,
                  -448.90467427, -448.79007768, -448.9659579,  -448.55792272, -448.75166424]
m_h_c_t_b_s_s = -448.84169460981764
d_h_c_t_b_s_s = 0.1504880123575198
```

### Tweak Book - Shifted Rosenbrock
```
p = 0.02
r = 10
v_h_c_t_b_s_ro = [1004.96216706, 1010.59483538, 1059.38723624, 1231.83968287, 1074.96875534,
                  1133.96758412, 1157.28613874, 1113.96419956,  947.78057586,  993.149034  ]
m_h_c_t_b_s_ro = 1072.790020917084
d_h_c_t_b_s_ro = 82.76332802349343
```

### Tweak Book - Shifted Rastring
```
p = 0.02
r = 10
v_h_c_t_b_s_ra = [-271.0592473, -271.18508148, -269.07648265, -268.94991295, -256.41134912,
                  -268.5656181, -276.32074544, -268.08940459, -275.73561079, -266.18099865]
m_h_c_t_b_s_ra = -269.1574451074545
d_h_c_t_b_s_ra = 5.2457228958423165
```

## Simulated Annealing

### Tweak Book - Shifted Sphere
```
p = 0.02
r = 10
v_s_a_t_b_s_s = [ -443.48464097, -443.3793431, -441.89325212, -443.19748406, -442.75356732,
                  -445.00679555, -443.78463621, -443.31465878, -443.84054266, -444.00018411]
m_s_a_t_b_s_s = -443.4655104881308
d_s_a_t_b_s_s = 0.7736388406486008
```

### Tweak Book - Shifted Rosenbrock
```
p = 0.02
r = 10
v_s_a_t_b_s_ro = [980.47697763, 1156.61962987, 1196.1478509,  1140.65640393, 1087.63932609,
                  1239.62264561, 1054.3994904,  1041.64757216, 1092.38879065, 1367.02406835]
m_s_a_t_b_s_ro = 1135.6622755574595
d_s_a_t_b_s_ro = 105.90673921783308
```

### Tweak Book - Shifted Rastring
```
p = 0.02
r = 10
v_s_a_t_b_s_ra = [-263.10954392, -261.68169129, -246.04764288, -251.97618544, -261.55005478,
                  -266.53617798, -252.56246195, -256.9262547,  -263.68921995, -263.71010443]
m_s_a_t_b_s_ra = -258.7789337326337
d_s_a_t_b_s_ra = 6.276157929401573
```

## Search Tabu

### Tweak Book - Shifted Sphere
```
p = 0.02
r = 10
t_s = 3
v_s_t_t_b_s_s = [ -447.82517356, -448.49152267, -448.6959678,  -448.47626668, -448.11314914,
                  -448.35987982, -448.35978147, -447.50214284, -448.66428286, -448.48921587]
m_s_t_t_b_s_s = -448.2977382705705
d_s_t_t_b_s_s = 0.3601532628835457
```

### Tweak Book - Shifted Rosenbrock
```
p = 0.02
r = 20
t_s = 3
v_s_t_t_b_s_ro = [2368.01326385, 10207.60724823, 2357.04728754, 4003.1352373, 3132.50728948,
                  2491.71171306, 2790.25082972, 2276.60372847, 2969.04384609, 5942.98167604]
m_s_t_t_b_s_ro = 3853.8902119768745
d_s_t_t_b_s_ro = 2366.0664510221663
```

### Tweak Book - Shifted Rastring
```
p = 0.02
r = 10
t_s = 3
v_s_t_t_b_s_ra = [-253.08769207, -242.63259014, -238.21162395, -240.86140322, -248.32401529,
                  -255.67282255, -255.24164896, -262.26210817, -246.59078615, -256.21777981]
m_s_t_t_b_s_ra = -249.91024702985314
d_s_t_t_b_s_ra = 7.41160793105595
```

## Evolution Strategy

### Tweak Book - Shifted Sphere
```
p = 0.08
r = 10
pa = 4
ch = 8
v_e_s_t_b_s_s = [ -398.61950041, -393.61337499, -391.48464849, -401.70388244, -392.67623487,
                  -386.26059611, -396.33062868, -387.56259527, -402.69792184, -382.80404004]
m_e_s_t_b_s_s = -393.3753423128192
d_e_s_t_b_s_s = 6.267297572944023
```

### Tweak Book - Shifted Rosenbrock
```
p = 0.02
r = 10
pa = 6
ch = 12
v_e_s_t_b_s_ro = [7173.24270992,  591.81246416, 6786.03367258,  585.39822934,  698.38219895,
                  761.6542849,  4081.48621717,  727.89032933,  782.91016796,  753.19015168]
m_e_s_t_b_s_ro = 2294.200042598193
d_e_s_t_b_s_ro = 2549.5393987820744
```

### Tweak Book - Shifted Rastring
```
p = 0.02
r = 10
pa = 6
ch = 12
v_e_s_t_b_s_ra = [-323.46413029, -324.96053899, -325.44402403, -323.24566281, -324.54282568,
                  -324.26446687, -323.85943724, -321.74907162, -323.51264082, -323.79603952]
m_e_s_t_b_s_ra = -323.8838837883337
d_e_s_t_b_s_ra = 0.9725918709844927
```

## Genetic Algorithm

### Tweak Book - Shifted Sphere
```
p = 0.02
r = 10
e_s = 2
pop_s = 8
to_s = 2
v_g_a_t_b_s_s = [ -449.90513437, -449.90470852, -449.91241588, -449.87719647, -449.93038364, 
                  -449.96038546, -449.94569847, -449.94933834, -449.95296278, -449.94310217]
m_g_a_t_b_s_s = -449.90596777416783
d_g_a_t_b_s_s = 0.017133479280194472
```

### Tweak Book - Shifted Rosenbrock
```
p = 0.02
r = 10
e_s = 2
pop_s = 4
to_s = 2
v_g_a_t_b_s_ro = [3.14401475e+10, 3.14315930e+10, 3.14358844e+10, 3.14550101e+10, 3.14464388e+10,
                  3.14246774e+10, 3.14225704e+10, 3.14275398e+10, 3.14215488e+10, 3.14205256e+10]
m_g_a_t_b_s_ro = 31423372379.49226
d_g_a_t_b_s_ro = 2495723.8339522546
```

### Tweak Book - Shifted Rastring
```
p = 0.02
r = 10
e_s = 6
pop_s = 12
to_s = 2
v_g_a_t_b_s_ra = [-327.52377503, -327.8927173,  -327.54166978, -327.42871595, -328.38497646, 
                  -327.84529736, -327.87146515, -327.51256102, -328.29303198, -327.62737614]
m_g_a_t_b_s_ra = -327.8299463288337
d_g_a_t_b_s_ra = 0.2676474921660463
```

# Testes Estatisticos

In [20]:
def test_statistical_all():
  v_h_c_t_b_s_s = [ -448.99958059, -449.08588058, -448.67478445, -448.8023115,  -448.88409216,
                    -448.90467427, -448.79007768, -448.9659579,  -448.55792272, -448.75166424]
  v_s_a_t_b_s_s = [ -443.48464097, -443.3793431, -441.89325212, -443.19748406, -442.75356732,
                    -445.00679555, -443.78463621, -443.31465878, -443.84054266, -444.00018411]
  v_s_t_t_b_s_s = [ -447.82517356, -448.49152267, -448.6959678,  -448.47626668, -448.11314914,
                    -448.35987982, -448.35978147, -447.50214284, -448.66428286, -448.48921587]
  v_e_s_t_b_s_s = [ -398.61950041, -393.61337499, -391.48464849, -401.70388244, -392.67623487,
                    -386.26059611, -396.33062868, -387.56259527, -402.69792184, -382.80404004]
  v_g_a_t_b_s_s = [ -449.90513437, -449.90470852, -449.91241588, -449.87719647, -449.93038364, 
                    -449.96038546, -449.94569847, -449.94933834, -449.95296278, -449.94310217]

  v_h_c_t_b_s_ro = [1004.96216706, 1010.59483538, 1059.38723624, 1231.83968287, 1074.96875534,
                    1133.96758412, 1157.28613874, 1113.96419956,  947.78057586,  993.149034  ]
  v_s_a_t_b_s_ro = [980.47697763, 1156.61962987, 1196.1478509,  1140.65640393, 1087.63932609,
                    1239.62264561, 1054.3994904,  1041.64757216, 1092.38879065, 1367.02406835]
  v_s_t_t_b_s_ro = [2368.01326385, 10207.60724823, 2357.04728754, 4003.1352373, 3132.50728948,
                    2491.71171306, 2790.25082972, 2276.60372847, 2969.04384609, 5942.98167604]
  v_e_s_t_b_s_ro = [7173.24270992,  591.81246416, 6786.03367258,  585.39822934,  698.38219895,
                    761.6542849,  4081.48621717,  727.89032933,  782.91016796,  753.19015168]
  v_g_a_t_b_s_ro = [3.14401475e+10, 3.14315930e+10, 3.14358844e+10, 3.14550101e+10, 3.14464388e+10,
                    3.14246774e+10, 3.14225704e+10, 3.14275398e+10, 3.14215488e+10, 3.14205256e+10]
                  
  v_h_c_t_b_s_ra = [-271.0592473, -271.18508148, -269.07648265, -268.94991295, -256.41134912,
                    -268.5656181, -276.32074544, -268.08940459, -275.73561079, -266.18099865]
  v_s_a_t_b_s_ra = [-263.10954392, -261.68169129, -246.04764288, -251.97618544, -261.55005478,
                    -266.53617798, -252.56246195, -256.9262547,  -263.68921995, -263.71010443]
  v_s_t_t_b_s_ra = [-253.08769207, -242.63259014, -238.21162395, -240.86140322, -248.32401529,
                    -255.67282255, -255.24164896, -262.26210817, -246.59078615, -256.21777981]
  v_e_s_t_b_s_ra = [-323.46413029, -324.96053899, -325.44402403, -323.24566281, -324.54282568,
                    -324.26446687, -323.85943724, -321.74907162, -323.51264082, -323.79603952]
  v_g_a_t_b_s_ra = [-327.52377503, -327.8927173,  -327.54166978, -327.42871595, -328.38497646, 
                    -327.84529736, -327.87146515, -327.51256102, -328.29303198, -327.62737614]

  data_sphere = np.array([v_h_c_t_b_s_s, v_s_a_t_b_s_s ,v_s_t_t_b_s_s, v_e_s_t_b_s_s, v_g_a_t_b_s_s])
  data_rosenbrock = np.array([v_h_c_t_b_s_ro, v_s_a_t_b_s_ro ,v_s_t_t_b_s_ro, v_e_s_t_b_s_ro, v_g_a_t_b_s_ro])
  data_rastring = np.array([v_h_c_t_b_s_ra, v_s_a_t_b_s_ra ,v_s_t_t_b_s_ra, v_e_s_t_b_s_ra, v_g_a_t_b_s_ra])

  print('SHIFTED SPHERE')
  test_statistical(data_sphere)
  print()
  print('SHIFTED ROSENBROCK')
  test_statistical(data_rosenbrock)
  print()
  print('SHIFTED RASTRING')
  test_statistical(data_rastring)
  print()